In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import numpy as np
import os
from utils.dataset import extract_data, get_poses, poses_2_archive

In [14]:
PATH='/docker_host'
DATA_PATH = os.path.join(PATH, 'data')
!ls {DATA_PATH}

ConvertHumanEva.m			data_3d_h36m.npz.sav
__init.py__				data_utils.py
convert_cdf_to_mat.m			detectron
data_2d_h36m_cpn_ft_h36m_dbb.npz	h36m.zip
data_2d_h36m_detectron_custom_h36m.npz	prepare_data_2d_h36m_generic.py
data_2d_h36m_detectron_ft_h36m.npz	prepare_data_2d_h36m_sh.py
data_2d_h36m_gt.npz			prepare_data_h36m.py
data_2d_h36m_sh_pt_mpii.npz		prepare_data_humaneva.py
data_3d_h36m.npz			videos


In [15]:
def import_detectron_poses(path):
    # Latin1 encoding because Detectron runs on Python 2.7
    data = np.load(path, encoding='latin1')
    kp = data['keypoints']
    bb = data['boxes']
    results = []
    for i in range(len(bb)):
        if len(bb[i][1]) == 0:
            assert i > 0
            # Use last pose in case of detection failure
            results.append(results[-1])
            continue
        best_match = np.argmax(bb[i][1][:, 4])
        keypoints = kp[i][1][best_match].T.copy()
        results.append(keypoints)
    results = np.array(results)
    # return results[:, :, 4:6] # Soft-argmax
    return results[:, :, [0, 1, 3]] # Argmax + score

In [17]:
def keypoints_2d_to_datasets(poses, subject='S1', action='Default'):
    """
    given a sequence of 2d keypoints for a single subject and action, 
    create the datasets necessary to predict 3d keypoints with Video2Pose run.py.
    """
    
    dataset_2d = dict({
        'positions_2d': dict({
            subject: dict({
                action: [poses] # array of cameras but we have only one camera
            })
        }),
        'metadata': dict({
            'layout_name': 'h36m', 
            'num_joints': 17, 
            'keypoints_symmetry': [[4, 5, 6, 11, 12, 13], [1, 2, 3, 14, 15, 16]]
        })
    })
    
    # to use run.py to generate 3d predictions, 
    # it requires a 3d dataset that matches 
    # all the subjects and actions in the 2d dataset
    dataset_3d_fake = dict({
        'positions_3d': dict({
            subject: dict({
                action: np.ones((poses.shape[0], 32, 3), dtype=np.float32)
            })
        })
    })
    
    return dataset_2d, dataset_3d_fake

In [18]:
def prepare_data_2d_from_detectron(detectron_data, output_file_2d, output_file_3d):
    keypoints_2d = import_detectron_poses(detectron_data)
    dataset_2d, dataset_3d = keypoints_2d_to_datasets(keypoints_2d)
    np.savez(output_file_2d, **dataset_2d)
    np.savez(output_file_3d, **dataset_3d)


In [19]:
detectron_file = os.path.join(DATA_PATH, 'detectron', 'keypoints.npz')
output_file_2d = os.path.join(DATA_PATH, 'data_2d_h36m_detectron_custom_h36m.npz')
output_file_3d = os.path.join(DATA_PATH, 'data_3d_h36m.npz')
!ls {detectron_file}

# from data.data_utils import import_detectron_poses

prepare_data_2d_from_detectron(detectron_file, output_file_2d, output_file_3d)

from common.h36m_dataset import Human36mDataset

dataset = Human36mDataset(output_file_3d)
print(type(dataset))

# print(f'kps.shape={kps.shape}')

/docker_host/data/detectron/keypoints.npz


IndexError: index 11 is out of bounds for axis 0 with size 5

In [4]:
# data_file = os.path.join(DATA_PATH, 'data_2d_h36m_detectron_ft_h36m.npz')
# positions_2d, metadata = extract_data(data_file)
# print(f'metadata={metadata}')
# poses = get_poses(positions_2d, action='Walking 1')
# # print(f'poses={type(poses)}')
# print(poses.shape) # [3000, 17, 2] 3000 frames, 17 keypoints/frame, 2 dims/keypoint (x,y)

metadata={'layout_name': 'h36m', 'num_joints': 17, 'keypoints_symmetry': [[4, 5, 6, 11, 12, 13], [1, 2, 3, 14, 15, 16]]}
(3477, 17, 2)


In [6]:
# test_poses = [
#     [[1,2], [3,4]]
# ]

# test_path = os.path.join(DATA_PATH, 'test1.npz')
# test_arch = poses_2_archive(test_poses)
# # 
# np.savez(test_path, **test_arch)

# positions_2d, metadata = extract_data(test_path)
# poses_round_trip = get_poses(positions_2d)
# # print(f'poses={type(poses)}')
# print(poses_round_trip) # [3000, 17, 2] 3000 frames, 17 keypoints/frame, 2 dims/keypoint (x,y)


[[[1, 2], [3, 4]]]
